#Import libraries

In [1]:
# Standard libraries
import math
import time
import random
from typing import List, Tuple, Optional, Union

# Numerical and GPU computation libraries
import numpy as np
from numba import cuda, jit, prange, float32

# Image processing libraries
import cv2
from PIL import Image

# Plotting libraries
import matplotlib.pyplot as plt

#Kernel definitions

##KMean

In [46]:
class KMeans:
    def __init__(self, n_clusters: int = 3, max_iter: int = 100, random_state: int = None):
        """
        Initialize K-Means Clustering

        Parameters:
        - n_clusters: Number of clusters
        - max_iter: Maximum iterations
        - random_state: Seed for reproducibility
        """
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = random_state

        # Attributes to be set during fitting
        self.centroids = None
        self.labels = None

    @cuda.jit
    def _calculate_distances_kernel(X, centroids, distances):

        idx = cuda.grid(1)  # Get the index of the current thread

        if idx < X.shape[0]:  # Ensure we are within bounds
            for j in range(centroids.shape[0]):
                dist = 0.0

                for k in range(X.shape[1]):
                    diff = X[idx, k] - centroids[j, k]
                    dist += diff * diff

                distances[idx, j] = math.sqrt(dist)  # Store the distance

    @cuda.jit
    def _update_centroids_kernel(X, labels, centroids, counts):

        idx = cuda.grid(1)

        if idx < centroids.shape[0]:  # Ensure we are within bounds
            for j in range(X.shape[1]):

                centroids[idx, j] = 0.0  # Reset centroids
            counts[idx] = 0  # Reset count

        # Accumulate sums
        for i in range(X.shape[0]):
            if labels[i] == idx:
                for j in range(X.shape[1]):

                    cuda.atomic.add(centroids[idx], j, X[i, j])  # Add to the centroid
                cuda.atomic.add(counts, idx, 1)  # Increment count for this centroid

    def fit(self, X: np.ndarray) -> 'KMeans':
        """
        Fit the K-Means algorithm

        Parameters:
        - X: Input data array

        Returns:
        - Self
        """
        print('k1')
        if self.random_state is not None:
            np.random.seed(self.random_state)

        # Randomly initialize centroids
        random_idx = np.random.choice(len(X), self.n_clusters, replace=False)
        self.centroids = X[random_idx]
        print('k2')
        # Allocate device memory
        d_X = cuda.to_device(X)
        d_centroids = cuda.to_device(self.centroids)
        d_labels = cuda.device_array(X.shape[0], dtype=np.int32)
        d_distances = cuda.device_array((X.shape[0], self.n_clusters), dtype=np.float32)
        print('k3')

        for _ in range(self.max_iter):

            threads_per_block = 256
            blocks_per_grid = (d_X.shape[0] + (threads_per_block - 1)) // threads_per_block

            print(f"Blocks per Grid: {blocks_per_grid}")
            print(f"Threads per Block: {threads_per_block}")
            print(f"d_X Shape: {d_X.shape}")
            print(f"d_Centroids Shape: {d_centroids.shape}")
            print(f"d_Distances Shape: {d_distances.shape}")

            print('k4')

            # Calculate distances
            self._calculate_distances_kernel[blocks_per_grid, threads_per_block](d_X, d_centroids, d_distances)
            cuda.synchronize()  # Wait for the kernel to finish
            print('k5')

            # Copy distances back to host
            distances = d_distances.copy_to_host()

            # Assign labels based on the closest centroid
            d_labels.copy_to_host(self.labels)  # Copy labels back to host

            # Update centroids
            counts = np.zeros(self.n_clusters, dtype=np.int32)
            d_counts = cuda.to_device(counts)
            self._update_centroids_kernel[blocks_per_grid, threads_per_block](d_X, d_labels, d_centroids, d_counts)
            cuda.synchronize()  # Wait for the kernel to finish
            print('k6')

            # Normalize centroids
            d_centroids.copy_to_host(self.centroids)  # Copy centroids back to host
            for i in range(self.n_clusters):
                if counts[i] > 0:
                    self.centroids[i] /= counts[i]  # Average the centroids

        return self


    def predict(self, X: np.ndarray) -> np.ndarray:

        # Allocate device memory
        d_X = cuda.to_device(X)
        d_centroids = cuda.to_device(self.centroids)
        d_distances = cuda.device_array((X.shape[0], self.n_clusters), dtype=np.float32)
        d_labels = cuda.device_array(X.shape[0], dtype=np.int32)

        threads_per_block = 256
        blocks_per_grid = (X.shape[0] + (threads_per_block - 1)) // threads_per_block

        print('p1')
        # Calculate distances
        self._calculate_distances_kernel[blocks_per_grid, threads_per_block](d_X, d_centroids, d_distances)
        cuda.synchronize()  # Wait for the kernel to finish

        print('p2')
        # Copy distances back to host
        distances = d_distances.copy_to_host()

        # Assign labels based on the closest centroid
        self.labels = np.argmin(distances, axis=1)

        return self.labels


##FuzzyCMean

In [3]:
class FuzzyCMeans:
    def __init__(self, n_clusters: int = 3, max_iter: int = 100,
                 m: float = 2.0, epsilon: float = 1e-4, random_state: int = None):
        """
        Initialize Fuzzy C-Means Clustering

        Parameters:
        - n_clusters: Number of clusters
        - max_iter: Maximum iterations
        - m: Fuzzy coefficient (default 2)
        - epsilon: Convergence threshold
        """

        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.m = m  # Fuzzy coefficient (should be > 1)
        self.epsilon = epsilon
        self.random_state = random_state

        # Attributes to be set during fitting
        self.centroids = None
        self.membership_matrix = None

    def _calculate_distances(self, X: np.ndarray, centroids: np.ndarray) -> np.ndarray:
        """
        Vectorized Euclidean distance calculation

        Parameters:
        - X: Input data points
        - centroids: Cluster centroids

        Returns:
        - Distance matrix
        """

        distances = np.sqrt(((X[:, np.newaxis, :] - centroids) ** 2).sum(axis=2))
        return distances

    def fit(self, X: np.ndarray) -> 'FuzzyCMeans':
        """
        Fit the Fuzzy C-Means algorithm

        Parameters:
        - X: Input data array

        Returns:
        - Self
        """

        n_samples, n_features = X.shape

        # Initialize random centroids
        if self.random_state is not None:
            np.random.seed(self.random_state)

        # Randomly initialize centroids
        random_idx = np.random.choice(len(X), self.n_clusters, replace=False)
        self.centroids = X[random_idx]

        # Initialize random membership matrix
        self.membership_matrix = np.random.dirichlet(np.ones(self.n_clusters), size=n_samples)

        for _ in range(self.max_iter):
            # Store previous centroids
            prev_centroids = self.centroids.copy()

            # Calculate centroids
            self.centroids = self._calculate_centroids(X)

            # Update membership matrix
            self._update_membership_matrix(X)

            # Check convergence
            if (prev_centroids is not None and
                np.allclose(self.centroids, prev_centroids, atol=self.epsilon)):
                break

        return self

    def _calculate_centroids(self, X: np.ndarray) -> np.ndarray:
        """
        Calculate cluster centroids based on the current membership matrix

        Parameters:
        - X: Input data array

        Returns:
        - Centroids array
        """

        # Calculate centroids as the weighted mean of all samples in the cluster
        weights = self.membership_matrix ** self.m
        centroids = (weights.T @ X) / weights.sum(axis=0)[:, np.newaxis]
        return centroids

    def _update_membership_matrix(self, X: np.ndarray):
        """
        Update membership matrix with vectorized calculation
            using NumPy's broadcasting and 'np.linalg.norm'

        Parameters:
        - X: Input data array
        """

        # Compute distances from each point to each centroid
        distances = np.linalg.norm(X[:, np.newaxis, :] - self.centroids, axis=2)

        # Avoid division by zero by setting small values to a minimum
        distances = np.maximum(distances, 1e-10)

        # Update membership matrix with broadcasting
        power = 2 / (self.m - 1)
        inv_distances = 1.0 / distances
        self.membership_matrix = inv_distances ** power / np.sum(inv_distances ** power, axis=1, keepdims=True)

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict hard cluster labels based on the highest membership values

        Parameters:
        - X: Input data array

        Returns:
        - Cluster labels
        """

        return np.argmax(self.membership_matrix, axis=1)

#Behaviors

In [39]:
class ImageClustering:
    def __init__(self, clustering_method, n_clusters=10):
        """
        Initialize Image Clustering

        Parameters:
        - clustering_method: KMeans or FuzzyCMeans class
        - n_clusters: Number of colors to reduce to
        """
        self.clustering_method = clustering_method
        self.n_clusters = n_clusters
        self.clusterer = None

    def load_image(self, image_path):
        """
        Load and preprocess image

        Parameters:
        - image_path: Path to the image file

        Returns:
        - Original image array
        - Flattened image data
        """
        try:
            # Open the image
            image = Image.open(image_path)

            # Convert to numpy array
            image_array = np.array(image)

            # Check if the image has 3 channels (RGB)
            if image_array.ndim != 3 or image_array.shape[2] != 3:
                raise ValueError("Image must have 3 color channels (RGB).")

            # Reshape image to 2D array of pixels
            pixels = image_array.reshape((-1, 3)).astype(np.float32)  # Use float32 for CUDA

            return image_array, pixels

        except Exception as e:
            raise IOError(f"Error loading image: {e}")

    def quantize_image(self, image_path):
        """
        Perform color quantization

        Parameters:
        - image_path: Path to the image file

        Returns:
        - Original image
        - Quantized image
        - Centroids (for cluster colors)
        - Labels (cluster assignments for each pixel)
        """

        # Load image
        original_image, pixels = self.load_image(image_path)

        try:
            # Start timing
            start_time = time.perf_counter()  # better
            print("1")

            # Perform clustering
            self.clusterer = self.clustering_method(n_clusters=self.n_clusters)
            print("111")
            self.clusterer.fit(pixels)
            print("11")

            # Get cluster centroids and labels for each pixel
            centroids = self.clusterer.centroids
            labels = self.clusterer.predict(pixels)
            print("2")
            # Reconstruct image using cluster centroids
            quantized_pixels = centroids[labels].astype('uint8')
            print("3")
            # Reshape back to original image shape
            quantized_image = quantized_pixels.reshape(original_image.shape)

            # End timing
            end_time = time.perf_counter()  # better

            print(f"Clustering Time: {end_time - start_time:.4f} seconds")

            # Debugging prints
            print(f"Original Image Shape: {original_image.shape}")
            print(f"Quantized Image Shape: {quantized_image.shape}")
            print(f"Centroids Shape: {centroids.shape}")
            print(f"Labels Shape: {labels.shape}")
            print(f"Pixels Shape: {pixels.shape}")

            return original_image, quantized_image, centroids, labels, pixels

        except Exception as e:
            raise RuntimeError(f"Error during color quantization: {e}")

    def visualize_results(self, original_image, quantized_image):
        """
        Visualize original and quantized images

        Parameters:
        - original_image: Original image array
        - quantized_image: Quantized image array
        """
        # Create figure with two subplots
        plt.figure(figsize=(12, 6))

        # Original Image
        plt.subplot(1, 2, 1)
        plt.title('Original Image')
        plt.imshow(original_image)
        plt.axis('off')

        # Quantized Image
        plt.subplot(1, 2, 2)
        plt.title(f'Quantized Image ({self.n_clusters} Colors)')
        plt.imshow(quantized_image)
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    def visualize_clusters(self, pixels, labels, centroids):
        """
        Visualize clusters in 2D space

        Parameters:
        - pixels: Original pixel data (flattened image)
        - labels: Cluster labels for each pixel
        - centroids: Centroid colors for each cluster
        - method: Dimensionality reduction method ('pca' or 'tsne')
        """

        # Reduce dimensions
        plt.figure(figsize=(8, 6))

        # Plot each cluster with its centroid color
        for i, color in enumerate(centroids):
            cluster_pixels = pixels[labels == i]

            # Using RGB color components for 2D plotting
            plt.scatter(cluster_pixels[:, 0], cluster_pixels[:, 1],
                        color=np.clip(color / 255, 0, 1),          # Normalize to [0,1] for matplotlib
                        label=f'Cluster {i}', s=10, alpha=0.6)

        # Mark the centroids in black
        plt.scatter(centroids[:, 0], centroids[:, 1], color='black', marker='x', s=100, label='Centroids')

        plt.title("Cluster Visualization with Exact Colors")
        plt.xlabel("Red channel")
        plt.ylabel("Green channel")
        plt.legend()
        plt.show()

    def save_quantized_image(self, quantized_image, output_path):
        """
        Save quantized image

        Parameters:
        - quantized_image: Quantized image array
        - output_path: Path to save the image
        """
        Image.fromarray(quantized_image).save(output_path)

#TEST

In [47]:
def main():
    # Image path
    image_path = '/content/wolf1.jpg'

    # Create image clustering instance
    image_clusterer = ImageClustering(
        clustering_method=KMeans,    # Choose KMeans or FuzzyCMeans
        n_clusters=10                     # Number of colors
    )

    try:
        # Perform color quantization
        original, quantized, centroids, labels, pixels = image_clusterer.quantize_image(image_path)

        # Visualize results and cluster colors in 2D space
        image_clusterer.visualize_results(original, quantized)
        image_clusterer.visualize_clusters(pixels, labels, centroids)

        # Optionally save quantized image
        image_clusterer.save_quantized_image(
            quantized,
            '/content/quantized_image.jpg'
        )


    except Exception as e:
        print(f"An error occurred: {e}")

# Run the main function
if __name__ == "__main__":
    main()

1
111
k1
k2
k3
Blocks per Grid: 2000
Threads per Block: 256
d_X Shape: (512000, 3)
d_Centroids Shape: (10, 3)
d_Distances Shape: (512000, 10)
k4
An error occurred: Error during color quantization: 'method' object is not subscriptable
